In [1]:
%env CUDA_VISIBLE_DEVICES=6

import math

import torch
from linear import HiggsLinear, pad_to_block

env: CUDA_VISIBLE_DEVICES=6


In [2]:
SIZE = 8192
HADAMARD_SIZE = 1024
GRID_DIM = 2

layer = HiggsLinear(
    SIZE, SIZE,
    GRID_DIM, 256,
    device="cuda",
    dtype=torch.float16,
)

In [3]:
input = torch.rand(1, 1, SIZE, device="cuda", dtype=torch.float16)

layer(input)

tensor([[[-5.3320, -5.3789, 15.3594,  ..., 16.2031,  3.0742, 31.0781]]],
       device='cuda:0', dtype=torch.float16)

In [4]:
from aqlm.utils import unpack_int_data, _dequantize_weight

codebooks = torch.load(f"../grids/EDEN{GRID_DIM}-256.pt").half()

post_hadamard_size = ((HADAMARD_SIZE - 1) // GRID_DIM + 1) * GRID_DIM

weight = _dequantize_weight(
    unpack_int_data(layer.codes, 8)[:,:,None],
    codebooks[None,:,None,:],
    # scales=layer.scales,
)

weight = weight.reshape(weight.shape[0], -1, post_hadamard_size) * layer.scales[...,None]

weight = weight.reshape(weight.shape[0], -1)

In [5]:
from fast_hadamard_transform import hadamard_transform

pad_to_block(hadamard_transform(input.reshape(-1, HADAMARD_SIZE), scale=1/math.sqrt(HADAMARD_SIZE)).reshape(1, -1), [-1], post_hadamard_size) @ weight.T

tensor([[-5.3125, -5.3867, 15.3594,  ..., 16.2031,  3.0820, 31.1250]],
       device='cuda:0', dtype=torch.float16)

In [6]:
%%time

with torch.inference_mode():
    for i in range(1000):
        layer(input)
        torch.cuda.synchronize()

CPU times: user 206 ms, sys: 808 µs, total: 207 ms
Wall time: 206 ms


In [7]:
%%time

with torch.inference_mode():
    for i in range(1000):
        torch.nn.functional.linear(input, weight)
        torch.cuda.synchronize()

CPU times: user 183 ms, sys: 0 ns, total: 183 ms
Wall time: 182 ms
